In [20]:
import pandas as pd
import numpy as np

In [2]:
posts = pd.read_csv('r_udub_posts.csv')
posts.head(5)

,title,selftext,link_flair_text,id,url,num_comments,score
0,Any UW redditors want to meet up Thursday 10/2...,"We failed on 10/22, but I think with a week of...",NaN,9y4hg,https://www.reddit.com/r/udub/comments/9y4hg/a...,6,4
1,We need a UW-ified logo.,If someone here has arcane skill in the graphi...,NaN,9ywtc,https://www.reddit.com/r/udub/comments/9ywtc/w...,2,3
2,Thursday bowling success!,[deleted],NaN,9z66c,https://www.reddit.com/r/udub/comments/9z66c/t...,2,3
3,"Next UW meetup: Thursday, 11/5 at 11:00am in t...",This time we will be playing ping pong followe...,NaN,a0ail,https://www.reddit.com/r/udub/comments/a0ail/n...,2,4
4,Next meetup: December 3rd. Need ideas,"Alright, so who is up for a December 3rd meetu...",NaN,a9lq8,https://www.reddit.com/r/udub/comments/a9lq8/n...,7,2


In [3]:
posts['link_flair_text'].unique()

array([nan, 'PSA', 'Rant', 'Random', 'Meme', 'Question', 'Discussion',
       'Academics', 'Student Life', 'Help', 'Event', 'Video',
       'Admissions', 'Advice', 'Poll', 'poll', 'No unrelated posts'],
      dtype=object)

# All Data

In [17]:
posts['combined_text'] = posts['title'] + " " + posts['selftext'].fillna("")
flair_categories = ["admissions", "academics", "student life", "advice", "discussion", "meme", "rant", "psa", "event", "poll"]

flairedNotSelf = posts[(posts['link_flair_text'].notnull()) & (posts['selftext'] != '[removed]') & (posts['selftext'] != '[deleted]') & posts['selftext'].notnull()]
ModelDataLower = flairedNotSelf.apply(lambda col: col.str.lower() if col.dtype == 'object' else col)
AllDataFilteredNoBody = ModelDataLower[ModelDataLower['link_flair_text'].isin(flair_categories)][['combined_text', 'link_flair_text']]

AllDataFilteredNoBody.groupby('link_flair_text').size().sort_values(ascending=False)

link_flair_text
advice          2360
academics       2100
student life    1551
admissions       977
discussion       865
poll             464
rant             388
psa              138
event            128
meme              60
dtype: int64

In [16]:
posts['combined_text'] = posts['title'] + " " + posts['selftext'].fillna("")
flair_categories = ["admissions", "academics", "student life", "advice", "discussion", "meme", "rant", "psa", "event", "poll"]

flairedNotSelf = posts[(posts['link_flair_text'].notnull()) & (posts['selftext'] != '[removed]') & (posts['selftext'] != '[deleted]') ]
ModelDataLower = flairedNotSelf.apply(lambda col: col.str.lower() if col.dtype == 'object' else col)
AllDataFiltered = ModelDataLower[ModelDataLower['link_flair_text'].isin(flair_categories)][['combined_text', 'link_flair_text']]

AllDataFiltered.groupby('link_flair_text').size().sort_values(ascending=False)

link_flair_text
advice          2468
academics       2270
student life    1870
admissions      1051
discussion      1030
meme             522
poll             471
rant             434
psa              260
event            208
dtype: int64

# 1000 post minimum

In [34]:
min_threshold = 1000
category_counts = AllDataFilteredNoBody.groupby('link_flair_text').size()
categories_to_keep = category_counts[category_counts >= min_threshold].index
df_Min_1000_NoBody = AllDataFilteredNoBody[AllDataFilteredNoBody['link_flair_text'].isin(categories_to_keep)]
df_Min_1000_NoBody.groupby('link_flair_text').size().sort_values(ascending=False)

link_flair_text
advice          2360
academics       2100
student life    1551
dtype: int64

In [35]:
min_threshold = 1000
category_counts = AllDataFiltered.groupby('link_flair_text').size()
categories_to_keep = category_counts[category_counts >= min_threshold].index
df_Min_1000_All = AllDataFiltered[AllDataFiltered['link_flair_text'].isin(categories_to_keep)]
df_Min_1000_All.groupby('link_flair_text').size().sort_values(ascending=False)

link_flair_text
advice          2468
academics       2270
student life    1870
admissions      1051
discussion      1030
dtype: int64

# 1000 post minimum balanced 

In [36]:
min_threshold = 1000
max_rows_per_category = 1000
category_counts = AllDataFilteredNoBody.groupby('link_flair_text').size()
categories_to_keep = category_counts[category_counts >= min_threshold].index
df_1000_NoBody = AllDataFilteredNoBody[AllDataFilteredNoBody['link_flair_text'].isin(categories_to_keep)]
for category in categories_to_keep:
    if len(df_1000_NoBody[df_1000_NoBody['link_flair_text'] == category]) > max_rows_per_category:
        df_1000_NoBody = df_1000_NoBody.drop(df_1000_NoBody[df_1000_NoBody['link_flair_text'] == category].sample(frac=1 - max_rows_per_category/len(df_1000_NoBody[df_1000_NoBody['link_flair_text'] == category]), random_state=1).index)

df_1000_NoBody.groupby('link_flair_text').size().sort_values(ascending=False)

link_flair_text
academics       1000
advice          1000
student life    1000
dtype: int64

In [38]:
min_threshold = 1000
max_rows_per_category = 1000
category_counts = AllDataFiltered.groupby('link_flair_text').size()
categories_to_keep = category_counts[category_counts >= min_threshold].index
df_1000_All = AllDataFiltered[AllDataFiltered['link_flair_text'].isin(categories_to_keep)]
for category in categories_to_keep:
    if len(df_1000_All[df_1000_All['link_flair_text'] == category]) > max_rows_per_category:
        df_1000_All = df_1000_All.drop(df_1000_All[df_1000_All['link_flair_text'] == category].sample(frac=1 - max_rows_per_category/len(df_1000_All[df_1000_All['link_flair_text'] == category]), random_state=1).index)

df_1000_All.groupby('link_flair_text').size().sort_values(ascending=False)

link_flair_text
academics       1000
admissions      1000
advice          1000
discussion      1000
student life    1000
dtype: int64

# 200 post minimum balanced

In [39]:
min_threshold = 200
max_rows_per_category = 200
category_counts = AllDataFilteredNoBody.groupby('link_flair_text').size()
categories_to_keep = category_counts[category_counts >= min_threshold].index
df_200_NoBody = AllDataFilteredNoBody[AllDataFilteredNoBody['link_flair_text'].isin(categories_to_keep)]
for category in categories_to_keep:
    if len(df_200_NoBody[df_200_NoBody['link_flair_text'] == category]) > max_rows_per_category:
        df_200_NoBody = df_200_NoBody.drop(df_200_NoBody[df_200_NoBody['link_flair_text'] == category].sample(frac=1 - max_rows_per_category/len(df_200_NoBody[df_200_NoBody['link_flair_text'] == category]), random_state=1).index)

df_200_NoBody.groupby('link_flair_text').size().sort_values(ascending=False)

link_flair_text
academics       200
admissions      200
advice          200
discussion      200
poll            200
rant            200
student life    200
dtype: int64

In [40]:
min_threshold = 200
max_rows_per_category = 200
category_counts = AllDataFiltered.groupby('link_flair_text').size()
categories_to_keep = category_counts[category_counts >= min_threshold].index
df_200_All = AllDataFiltered[AllDataFiltered['link_flair_text'].isin(categories_to_keep)]
for category in categories_to_keep:
    if len(df_200_All[df_200_All['link_flair_text'] == category]) > max_rows_per_category:
        df_200_All = df_200_All.drop(df_200_All[df_200_All['link_flair_text'] == category].sample(frac=1 - max_rows_per_category/len(df_200_All[df_200_All['link_flair_text'] == category]), random_state=1).index)

df_200_All.groupby('link_flair_text').size().sort_values(ascending=False)

link_flair_text
academics       200
admissions      200
advice          200
discussion      200
event           200
meme            200
poll            200
psa             200
rant            200
student life    200
dtype: int64

In [41]:
csv_file_path = 'AllDataFilteredNoBody.csv'
AllDataFilteredNoBody.to_csv(csv_file_path, index=False)
print("DataFrame saved as CSV:", csv_file_path) 

DataFrame saved as CSV: AllDataFilteredNoBody.csv


In [42]:
csv_file_path = 'AllDataFiltered.csv'
AllDataFiltered.to_csv(csv_file_path, index=False)
print("DataFrame saved as CSV:", csv_file_path) 

DataFrame saved as CSV: AllDataFiltered.csv


In [43]:
csv_file_path = 'df_Min_1000_NoBody.csv'
df_Min_1000_NoBody.to_csv(csv_file_path, index=False)
print("DataFrame saved as CSV:", csv_file_path) 

DataFrame saved as CSV: df_Min_1000_NoBody.csv


In [44]:
csv_file_path = 'df_Min_1000_All.csv'
df_Min_1000_All.to_csv(csv_file_path, index=False)
print("DataFrame saved as CSV:", csv_file_path) 

DataFrame saved as CSV: df_Min_1000_All.csv


In [45]:
csv_file_path = 'df_1000_NoBody.csv'
df_1000_NoBody.to_csv(csv_file_path, index=False)
print("DataFrame saved as CSV:", csv_file_path) 

DataFrame saved as CSV: df_1000_NoBody.csv


In [46]:
csv_file_path = 'df_1000_All.csv'
df_1000_All.to_csv(csv_file_path, index=False)
print("DataFrame saved as CSV:", csv_file_path) 

DataFrame saved as CSV: df_1000_All.csv


In [47]:
csv_file_path = 'df_200_NoBody.csv'
df_200_NoBody.to_csv(csv_file_path, index=False)
print("DataFrame saved as CSV:", csv_file_path) 

DataFrame saved as CSV: df_200_NoBody.csv


In [48]:
csv_file_path = 'df_200_All.csv'
df_200_All.to_csv(csv_file_path, index=False)
print("DataFrame saved as CSV:", csv_file_path) 

DataFrame saved as CSV: df_200_All.csv
